In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import (
    GPT2TokenizerFast,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch
import hopsworks
import numpy as np
import evaluate

c:\Users\dhanu\anaconda3\envs\fin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/text-embedding-ada-002')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT4Tokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def get_decoding(dataset, embedding_object):
    decodings = []
    
    for data in dataset["embeddings"]:
      
        decoded_text = embedding_object.decode(convstr2vec(data))
        
        decodings.append(decoded_text)

    dataset_decoded = dataset.copy()
    dataset_decoded["text"] = decodings
    dataset_decoded = dataset_decoded.drop(columns=["embeddings"])
    return dataset_decoded
    
def convstr2vec(temp):
    lst=temp.split(",")
    lst[0]=lst[0].replace("[","")
    lst[-1]=lst[-1].replace("]","")
    lst=[int(i) for i in lst]
    return lst

In [4]:
hopsworks_project = hopsworks.login() 
fs = hopsworks_project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/546965
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
training_fg = fs.get_or_create_feature_group("news_sentiment_traindata", version=1)
test_fg = fs.get_or_create_feature_group("news_sentiment_testdata", version=1)
# get all of the data from the feature group
training_features = training_fg.read()
testing_features = test_fg.read()

Finished: Reading data from Hopsworks, using Hive (32.86s) 
Finished: Reading data from Hopsworks, using Hive (4.82s) 


In [9]:
training_data = get_decoding(training_features, tokenizer)
testing_data = get_decoding(testing_features, tokenizer)

In [10]:
# print the 50th row
print(training_data.iloc[50])

label                                                    2
text     Edited Transcript of FHER3.SA earnings confere...
Name: 50, dtype: object


In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


In [12]:
# Convert pandas dataframes to Hugging Face datasets
train_dataset = Dataset.from_pandas(training_data)
test_dataset = Dataset.from_pandas(testing_data)

# Combine datasets into a DatasetDict
datasets = DatasetDict({'train': train_dataset, 'test': test_dataset})

In [13]:
tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map: 100%|██████████| 3077/3077 [00:01<00:00, 2097.90 examples/s]


In [14]:
def get_compute_metrics(metric):
    def compute_metrics(eval_pred): 
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)
    return compute_metrics

In [15]:
id2label = {0: "Negative", 1: "Positive", 2: "Neutral"}
label2id = {val: key for key, val in id2label.items()}

def model_init():
    return AutoModelForSequenceClassification.from_pretrained('bert-base-cased', return_dict=True, num_labels=3,
                                                             id2label=id2label, label2id=label2id)

metric = evaluate.load("accuracy")
compute_metrics = get_compute_metrics(metric)
training_args = TrainingArguments(
    output_dir="bert_sentiment_trainer", 
    evaluation_strategy="steps",
    num_train_epochs=8,
    save_total_limit=3,
    seed=42,
    lr_scheduler_type='constant_with_warmup',
    warmup_steps=50,
    max_steps=3000,
    save_strategy="steps",
    save_steps=250,
    fp16=False,
    eval_steps=250,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

tokenized_train_dataset = tokenized_datasets["train"].shuffle(seed=55)
tokenized_test_dataset = tokenized_datasets["test"].shuffle(seed=55)

trainer = Trainer(
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    model_init=model_init,
    tokenizer=tokenizer,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [4, 8, 16]),
    }
 
best_trials = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=10,
)

print(best_trials)

[I 2024-03-12 10:33:41,640] A new study created in memory with name: no-name-2d4c7f89-ac60-40e9-933f-fcb12d9d138f
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 5/3000 [06:23<63:28:37, 76.30s/it][W 2024-03-12 10:41:15,690] Trial 0 failed with parameters: {'learning_rate': 1.5246779762382621e-05, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dhanu\anaconda3\envs\fin\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\dhanu\anaconda3\envs\fin\lib\site-packages\transformers\integrations\integration_utils.py", line 199, in _objective
    tr

KeyboardInterrupt: 